# Method of finite differences
The first thing to be understood is that ARS is a shallow layering algorithm where we're only going to approximate gradients, differing a lot to how we'd calcualte gradients in traditional gradient descent.

**Before we'd say that:**  
$\hspace{2cm} \Delta w = -\alpha \dfrac{\partial \epsilon}{\partial w_{ij}}$
$\hspace{5.3cm}$ *Where epsilon = error function, alpha = learning rate and w = weight*  
  
  
**But now:**  
$\hspace{2cm} \Delta w \approx -\alpha \dfrac{
                                                \epsilon(w_{ij} + pert_{ij}) - \epsilon(w_{ij})
                                             }{
                                                 pert_{ij}
                                             }$
$\hspace{2cm}$ Where $pert_{ij}$ = the perturbed weight connecting the $i^{th}$ neuron to the $j^{th}$ neuron.  

**as**  
$\hspace{2cm} f^\prime(x) \approx \lim\limits_{h\to 0} \dfrac{f(x + h) - f(x)}{h}$  
  
So as you can see, the error function is now esimated through the incremental ratio shown above as we also want the perturbed weights, $(pert_{ij})$ to approach 0.  
  
Now suppose we had a perceptron where with:  
- 3 input values
- 2 output values
- Therefore 3 x 2 = 6 weights between the 6 synaptic connections

**Our matrix of weights will then be shaped as:**  

$\begin{bmatrix}
    w_{1,1} & w_{1,2} \\
    w_{2,1} & w_{2,2} \\
    w_{3,1} & w_{3,2} \\
\end{bmatrix}$  
  
**And our matrix of positively perturbed weights will be:**  

$\begin{bmatrix}
    w_{1,1} + \sigma p, & w_{1,2} + \sigma p \\
    w_{2,1} + \sigma p, & w_{2,2} + \sigma p \\
    w_{3,1} + \sigma p, & w_{3,2} + \sigma p \\
\end{bmatrix}$  
  
**With negatively perturbed weights being:**  

$\begin{bmatrix}
    w_{1,1} - \sigma p, & w_{1,2} - \sigma p \\
    w_{2,1} - \sigma p, & w_{2,2} - \sigma p \\
    w_{3,1} - \sigma p, & w_{3,2} - \sigma p \\
\end{bmatrix}$  
$\hspace{2cm}$*p is a random number between 0 and 1, pertubing the weights and sigma is the exploration noise.*    
In this program we'll generate 16 instances of positively and negitively purturbed weights.